<a href="https://colab.research.google.com/github/JN37/ML_Models/blob/master/Rag_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu langchain scikit-learn pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━

In [2]:
# Make sure UTF-8 locale to install LangChain
import locale

locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
# Load PDFs
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/enphase_5p_iq_battery.pdf")
docs = loader.load_and_split()

In [ ]:
# Load document from web
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://www.svd.se/a/4on1nq/ny-konstillhorighetslag-aktuell-i-svensk-politik-och-i-tyskland",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

Set up the text splitter (needed to search the vector space of embeddings)


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# If chunk lengths are not aligned with embedding model limit, it might be a good idea to split according to tokens instead. If so, use RecursiveCharacterTextSplitter.from_huggingface_tokenizer

chunk_size = 300
chunk_overlap = 30

splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunked_docs = splitter.split_documents(docs)

Set up embeddings model using the HuggingFace hub

In [6]:
!pip install huggingface_hub


In [7]:
# Load vector database embeddings model
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

vector_db = FAISS.from_documents(chunked_docs, embeddings)

# embeddings = HuggingFaceEmbeddings(
#     model_name="BAAI/bge-base-en-v1.5",
#     encode_kwargs={"normalize_embeddings": True} # True uses cosine similarity
#     )

# # Normalize the data in the vector database to improve the search

# vector_db = FAISS.from_documents(
#     chunked_docs,
#     embeddings,
#     distance_strategy=DistanceStrategy.COSINE
#   )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Setting up retrievers from FAISS lib

In [8]:
# Test with both similarity_search and max_marginal_relevance methods
k = 3 # Number of results
sim_retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": k}) # Euclidean Distance search
mmr_retriever = vector_db.as_retriever(search_type="mmr", search_kwargs={"k": k}) # MMR aims to select a set of documents that are not only relevant to the query but also diverse from each other.

# It is also possible to use a TFIDFRetriever that does not use embeddings

Set up LLM with quantization. Quantization is a technique used to compress neural network models by representing weights with fewer bits

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "HuggingFaceH4/zephyr-7b-beta"

# Quantize model (represent weights in lower bits) to speed up inference
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Without Quantization
# model = AutoModelForCausalLM.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Set up the text generation pipeline

In [10]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

Set up the prompt format, the LLM chain and the RAG chain

In [11]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

print(prompt)

input_variables=['context', 'question'] template='\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n{context}\n\n</s>\n<|user|>\n{question}\n</s>\n<|assistant|>\n\n '


Set up LLM- and RAG Chains

In [13]:
# LLM Chain (without RAG)
llm_chain = prompt | llm | StrOutputParser()

# Choose a retriever
retriever = sim_retriever
# retriever = mmr_retriever

# RAG Chain (with the retriever connected to the vector database) combined with LLM Chain
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

In [14]:
question = "How long is the warranty of the Enphase battery?"

Test the LLM chain

In [15]:
llm_chain.invoke({"context": "", "question": question})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


" The length of the warranty for Enphase batteries may vary depending on the specific model and region, but generally, Enphase offers a standard warranty of 10 years for their lithium-ion batteries. This warranty covers defects in materials and workmanship, as well as capacity loss under normal operating conditions. It's always best to check the exact terms and conditions of the warranty for your specific product and location with Enphase or an authorized reseller."

Test the RAG chain

In [16]:
rag_chain.invoke(question)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


' According to the provided context, the Enphase battery comes with a 15-year limited warranty. This information can be found in the metadata of one of the Documents, which indicates that it applies to "of prior generations of IQ Battery" as well as the specific model mentioned in another Document ("IQ8D-BAT"). Therefore, we can assume that this warranty applies to the Enphase battery being discussed in the initial Document as well.'

**EVALUATION**

After training a LLM from scratch, they are evaluated on standard benchmarks, such as:

1. GLUE (https://gluebenchmark.com/)
2.

In [20]:
# Prepare docs for generating QA examples
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in docs:
    docs_processed += text_splitter.split_documents([doc])

Set up agents for generating questions

In [24]:
from huggingface_hub import InferenceClient
import json

qa_model = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=qa_model,
    timeout=120,
)


def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 1000},
            "task": "text-generation",
        },
    )
    return json.loads(response.decode())[0]["generated_text"]


call_llm(llm_client, "This is a test context")

'This is a test context for the `@mui/material` library.\n\n## Installation\n\n```sh\nnpm install @mui/material\n```\n\n## Usage\n\n```jsx\nimport React from \'react\';\nimport { Button } from \'@mui/material\';\n\nfunction App() {\n  return (\n    <div className="App">\n      <Button variant="contained" color="primary">\n        Hello World\n      </Button>\n    </div>\n  );\n}\n\nexport default App;\n```\n\n## Documentation\n\n- [Material-UI](https://material-ui.com/)\n- [Material Design](https://material.io/)'

Set up the Generation Prompt

In [25]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

Generate QA samples

In [ ]:
import random
from tqdm import tqdm

n_generations = 1

print(f"Generating {n_generations} QA samples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, n_generations)):
    # Generate QA couple
    output_QA_couple = call_llm(llm_client, QA_generation_prompt.format(context=sampled_context.page_content))
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

In [ ]:
display(pd.DataFrame(outputs).head(1))